<a href="https://colab.research.google.com/github/charan-d55/Vector-Semantics-Evaluation/blob/main/Vector_Semantics_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install pytrec_eval

In [14]:
import nltk
nltk.download('brown')


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [15]:
def data():
    brownDS = brown.sents(categories=['news', 'hobbies'])
    dd =[]
    for i in brownDS:
        dd.append(" ".join(i))
    return dd
C = data()

IMporting all used libraries

In [16]:
import numpy as np
import pandas as pd
import copy
import pytrec_eval
import json
import matplotlib.pyplot as plt

In [17]:
from nltk.corpus import brown, wordnet
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from sklearn.metrics.pairwise import cosine_similarity as cs

Reading and storing the gold standard dataset i.e SimLex-999

In [18]:
C = data()
f = open("SimLex-999.txt", encoding = 'utf-8')
W1 = []
W2 = []
Sim = []
for line in f.readlines()[1:]:
    fields = line.strip().split('\t')
    W1.append(fields[0].lower())
    W2.append(fields[1].lower())
    Sim.append(float(fields[3]))
f.close()
W1 = np.asarray(W1)
W2 = np.asarray(W2)
Sim = np.asarray(Sim)

Method for Transitive Analysis

In [19]:
def transitive(topk,W1,W2,Sim,vocab):
    for word in topk.keys():
        if len(topk[word]) >= 10:
            topk[word] = sorted(topk[word], reverse=True)[:10]
        else:
            topk[word] = sorted(topk[word], reverse=True)
            tw = np.asarray(topk[word])
            addedWords = []
            for sim,word2 in topk[word]:
                if word2 in W1:
                    indices = [i for i, x in enumerate(W1) if x == word2]
                    for ind in indices:
                        if W2[ind] not in tw[:,1] and W2[ind]!=word and W2[ind] not in addedWords:
                            addedWords.append([Sim[ind]*(sim/max(Sim)),W2[ind]])
                if word2 in W2:
                    indices = [i for i, x in enumerate(W2) if x == word2]
                    for ind in indices:
                        if W1[ind] not in tw[:,1] and W1[ind]!=word and W1[ind] not in addedWords:
                            addedWords.append([Sim[ind]*(sim/max(Sim)),W1[ind]])
            addCounter = 0
            addedWords = sorted(addedWords, reverse=True)
            while len(topk[word])<10 and addCounter<len(addedWords):
                topk[word].append(addedWords[addCounter])
                addCounter+=1
    return topk

def transitivityAnalysis(topk,W1,W2,Sim,vocab, render=False):
    lens = []
    lens.append(notten(topk))
    topk = transitive(topk,W1,W2,Sim,vocab)
    while notten(topk) != lens[-1]:
        lens.append(notten(topk))
        topk = transitive(topk,W1,W2,Sim,vocab)
    if render:
        plt.plot(range(len(lens)),lens)
        plt.ylabel('# similar words != 10')
        plt.xlabel('# transitive operations')
        plt.title('# required transitive operations')
        plt.show()
    return topk

def notten(topk):
    count = 0
    for i in topk.keys():
        if len(topk[i]) != 10:
            count+=1
    return count

Method to gather topk in gold standard SimLex-999

In [20]:
def golden_top_K(W1,W2,Sim,vocab):
    topk = {}
    for i in range(len(W1)):
        if W1[i] in vocab:
            first = W1[i]
            second = W2[i]
            if first not in topk.keys():
                topk[first] = [[Sim[i], second]]
            else:
                topk[first].append([Sim[i], second])
        if W2[i] in vocab:
            first = W2[i]
            second = W1[i]
            if first not in topk.keys():
                topk[first] = [[Sim[i], second]]
            else:
                topk[first].append([Sim[i], second])
    return topk

In [21]:
def DictonaryGeneration(model,G):
    M = {}
    for k in model.keys():
        M[k] = {}
        for v in model[k]:
            M[k][v] = 1
    GG = {}
    for k in G.keys():
        GG[k] = {}
        for v in G[k]:
            GG[k][v[1]] = 1
    return M,GG
def DictonaryGenerationW2V(model,G):
    M = {}
    for k in model.keys():
        M[k] = {}
        for v in model[k]:
            M[k][v[0]] = 1
    GG = {}
    for k in G.keys():
        GG[k] = {}
        for v in G[k]:
            GG[k][v[1]] = 1
    return M,GG


Method to gather top 10 similar words in both TF-iDF and Word2Vec

In [22]:
def search(GT, similarities, vocab):
    TOPS = {}
    vv = np.asarray(vocab)
    for i in G.keys():
        if i in vv:
            ind = vocab.index(i)
            simVec = similarities[ind]
            inds = np.flip(simVec.argsort()[-11:])
            inds = np.setdiff1d(inds,ind)
            tops = vv[np.array(inds)]
            TOPS[i] = tops
    return TOPS
def searchW2V(model,G):
    TOPS = {}
    for i in G.keys():
        if model.wv.has_index_for(i):
            sims = model.wv.most_similar(i,topn=10)
            TOPS[i] = sims
    return TOPS


Method for evaluation

In [23]:
def evaluatorfunc(model,truth,method):
    if method=='TF-iDF':
        q, run = DictonaryGeneration(model, truth)
    elif method=='Word2Vec':
        q, run = DictonaryGenerationW2V(model, truth)
    evaluator = pytrec_eval.RelevanceEvaluator(q, {f'ndcg'})
    a = json.dumps(evaluator.evaluate(run), indent=1)
    b = evaluator.evaluate(run)
    return b

Change the value of method variable to TF-iDF or Word2Vec to get mean nDCG

In [24]:
method = 'Word2Vec'
if method == 'TF-iDF':
    tfIdfVectorizer=tfidf(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(C)
    d = pd.DataFrame(tfIdf.T.todense())
    similarities = cs(d)
    keys = tfIdfVectorizer.vocabulary_.keys()
    vocab = []
    for i in keys:
        vocab.append(i)
    gg = golden_top_K(W1,W2,Sim,vocab)
    G = transitivityAnalysis(copy.deepcopy(gg),W1,W2,Sim,vocab,render=False)
    modelTops = search(G, similarities, vocab)
    a = evaluatorfunc(modelTops,G,method)
    d1 = []
    for i in a.keys():
        d1.append(a[i]['ndcg'])
    print(f'mean ndcg:',sum(d1)/len(d1))

elif method == 'Word2Vec':
    results = []
    window_size = [1, 2, 5, 10]
    vector_size = [10, 50, 100, 300]
    for ws in window_size:
        for vs in vector_size:
            model = Word2Vec(sentences=brownDS, vector_size=vs, window=ws, workers=4, epochs=30)
            vocab = list(model.wv.index_to_key)
            gg = golden_top_K(W1,W2,Sim,vocab)
            G = transitivityAnalysis(copy.deepcopy(gg),W1,W2,Sim,vocab)
            W2Vtops = searchW2V(model,G)
            results = evaluatorfunc(W2Vtops,G,method)
            d1 = []
            for i in results.keys():
                d1.append(results[i]['ndcg'])
            print(f'mean ndcg ws:{ws}vs:{vs}:',sum(d1)/len(d1))

mean ndcg ws:1,vs:10: 0.012483483949684415
mean ndcg ws:1,vs:50: 0.012834164991244793
mean ndcg ws:1,vs:100: 0.012630419877974391
mean ndcg ws:1,vs:300: 0.013412902678666903
mean ndcg ws:2,vs:10: 0.010577330790196083
mean ndcg ws:2,vs:50: 0.015212929863626686
mean ndcg ws:2,vs:100: 0.014905052787661077
mean ndcg ws:2,vs:300: 0.013891390469780088
mean ndcg ws:5,vs:10: 0.008972266204701613
mean ndcg ws:5,vs:50: 0.01786857254310043
mean ndcg ws:5,vs:100: 0.018386386898333565
mean ndcg ws:5,vs:300: 0.01835527689849163
mean ndcg ws:10,vs:10: 0.008113602575895672
mean ndcg ws:10,vs:50: 0.01365907585649031
mean ndcg ws:10,vs:100: 0.01568093860609313
mean ndcg ws:10,vs:300: 0.014134436042879254
